In [ ]:
from lib.model_utils import FeatureConfig, ModelConfig, evaluate_model, MissingValueConfig, forecast_bias, plot_forecast
from lib.feature_engineering_utils import LogTime
from lib.utils import format_plot
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import mean_squared_error as mse
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
from pathlib import Path
from sklearn.linear_model import LassoCV
from xgboost import XGBRFRegressor
from lightgbm import LGBMRegressor
from tqdm.autonotebook import tqdm
import plotly.io as pio
pio.templates.default = "plotly_white"
np.random.seed(42)
tqdm.pandas()

/home/hien/Work/AI/london_smart_meters/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%cd ../..
project_folder = Path("AI/london_smart_meters")
source_data = project_folder/"data"
preprocessed_data = project_folder/"output"/"preprocessed_data"
dst_img = project_folder/"output"/"img"
dst_result = project_folder/"output"/"result"

/home/hien/Work


In [ ]:
train_df = pd.read_parquet(preprocessed_data/"selected_blocks_train_missing_imputed_feature_engg.parquet")
val_df = pd.read_parquet(preprocessed_data/"selected_blocks_val_missing_imputed_feature_engg.parquet")
test_df = pd.read_parquet(preprocessed_data/"selected_blocks_test_missing_imputed_feature_engg.parquet")

In [ ]:
baseline_metrics_df = pd.read_pickle(dst_result/"baseline_test_metrics_df.pkl")
baseline_aggregate_metrics_df = pd.read_pickle(dst_result/"baseline_test_aggregate_metrics.pkl")

In [5]:
baseline_metrics_df

,Algorithm,MSE,RMSE,MAE,Forecast Bias
0,AutoETS,0.058671,0.242221,0.119127,-10.707988
1,ARIMA,0.069428,0.263492,0.132783,-13.314743


In [6]:
baseline_aggregate_metrics_df

,mean_absolute_error,mean_squared_error,root_mean_squared_error,forecast_bias,LCLid,Time Elapsed,Model
0,0.063103,0.014058,0.118568,13.096588,MAC003069,333.531509,AutoETS
1,0.031380,0.002216,0.047077,-17.411985,MAC002220,333.531509,AutoETS
2,0.118270,0.054404,0.233246,-11.551055,MAC004338,333.531509,AutoETS
3,0.049294,0.009882,0.099409,-1.060129,MAC001807,333.531509,AutoETS
4,0.088535,0.028487,0.168780,22.990714,MAC001985,333.531509,AutoETS
...,...,...,...,...,...,...,...
365,0.133915,0.049777,0.223108,26.121258,MAC003784,915.781631,ARIMA
366,0.094975,0.034475,0.185676,19.601429,MAC001453,915.781631,ARIMA
367,0.055108,0.012946,0.113781,2.460679,MAC000909,915.781631,ARIMA
368,0.174382,0.063694,0.252378,35.041399,MAC002235,915.781631,ARIMA


In [7]:
train_df = pd.concat([train_df, val_df])
del val_df
len(train_df.LCLid.unique())

185

In [8]:
train_df.columns

Index(['timestamp', 'LCLid', 'frequency', 'timestamp_Week', 'series_length',
       'stdorToU', 'Acorn', 'Acorn_grouped', 'file', 'holidays', 'visibility',
       'windBearing', 'temperature', 'dewPoint', 'pressure',
       'apparentTemperature', 'windSpeed', 'precipType', 'icon', 'humidity',
       'summary', 'hour', 'weekday', 'day_hourly_profile',
       'energy_consumption', 'energy_consumption_lag_1',
       'energy_consumption_lag_2', 'energy_consumption_lag_3',
       'energy_consumption_lag_4', 'energy_consumption_lag_5',
       'energy_consumption_lag_46', 'energy_consumption_lag_47',
       'energy_consumption_lag_48', 'energy_consumption_lag_49',
       'energy_consumption_lag_50', 'energy_consumption_lag_334',
       'energy_consumption_lag_335', 'energy_consumption_lag_336',
       'energy_consumption_lag_337', 'energy_consumption_lag_338',
       'timestamp_Month', 'timestamp_Quarter', 'timestamp_Is_quarter_end',
       'timestamp_Is_quarter_start', 'timestamp_Is_year_end

# Defining Columns Data

In [9]:
feat_config = FeatureConfig(
    date="timestamp",
    target="energy_consumption",
    continuous_features=[
        "visibility",
        "windBearing",
        "temperature",
        "dewPoint",
        "pressure",
        "apparentTemperature",
        "windSpeed",
        "humidity",
        "energy_consumption_lag_1",
        "energy_consumption_lag_2",
        "energy_consumption_lag_3",
        "energy_consumption_lag_4",
        "energy_consumption_lag_5",
        "energy_consumption_lag_46",
        "energy_consumption_lag_47",
        "energy_consumption_lag_48",
        "energy_consumption_lag_49",
        "energy_consumption_lag_50",
        "energy_consumption_lag_334",
        "energy_consumption_lag_335",
        "energy_consumption_lag_336",
        "energy_consumption_lag_337",
        "energy_consumption_lag_338",
        "timestamp_Elapsed",
        "day_hourly_profile",
    ],
    categorical_features=[
        "holidays",
        "precipType",
        "icon",
        "summary",
        "timestamp_Month",
        "timestamp_Quarter",
        "timestamp_Week",
        "timestamp_Day",
        "timestamp_Dayofweek",
        "timestamp_Dayofyear",
        "timestamp_Hour",
        "timestamp_Minute",
        "weekday",
        "hour",
    ],
    boolean_features=[
        "timestamp_Is_quarter_end",
        "timestamp_Is_quarter_start",
        "timestamp_Is_year_end",
        "timestamp_Is_year_start",
        "timestamp_Is_month_start",
    ],
    index_cols=["timestamp"],
    exogenous_features=[
        "holidays",
        "precipType",
        "icon",
        "summary",
        "visibility",
        "windBearing",
        "temperature",
        "dewPoint",
        "pressure",
        "apparentTemperature",
        "windSpeed",
        "humidity",
    ],
)

In [10]:
nc  = train_df.isnull().sum()
nc[nc>0]

pressure                       3844
energy_consumption_lag_1        185
energy_consumption_lag_2        370
energy_consumption_lag_3        555
energy_consumption_lag_4        740
energy_consumption_lag_5        925
energy_consumption_lag_46      8510
energy_consumption_lag_47      8695
energy_consumption_lag_48      8880
energy_consumption_lag_49      9065
energy_consumption_lag_50      9250
energy_consumption_lag_334    61790
energy_consumption_lag_335    61975
energy_consumption_lag_336    62160
energy_consumption_lag_337    62345
energy_consumption_lag_338    62530
dtype: int64

In [11]:
nc = test_df.isnull().sum()
nc[nc>0]

Series([], dtype: int64)

In [12]:
missing_value_config = MissingValueConfig(
    bfill_columns=[
        "pressure",
        "energy_consumption_lag_1",
        "energy_consumption_lag_2",
        "energy_consumption_lag_3",
        "energy_consumption_lag_4",
        "energy_consumption_lag_5",
        "energy_consumption_lag_46",
        "energy_consumption_lag_47",
        "energy_consumption_lag_48",
        "energy_consumption_lag_49",
        "energy_consumption_lag_50",
        "energy_consumption_lag_334",
        "energy_consumption_lag_335",
        "energy_consumption_lag_336",
        "energy_consumption_lag_337",
        "energy_consumption_lag_338",
    ],
    ffill_columns=[],
    zero_fill_columns=[],
)

# ML Forecast

## Training and Evaluating Models

In [13]:
lcl_ids = sorted(train_df.LCLid.unique())
models_to_run = [
    ModelConfig(
        model=LassoCV(), name="Lasso Regression", normalize=True, fill_missing=True
    ),
    ModelConfig(
        model=XGBRFRegressor(random_state=42, max_depth=4),
        name="XGB Random Forest",
        normalize=False,
        fill_missing=False,
    ),
    ModelConfig(
        model=LGBMRegressor(random_state=42),
        name="LightGBM",
        normalize=False,
        fill_missing=False,
    ),
]

In [14]:
all_preds = []
all_metrics = []

# We can parallelize this loop to run this faster
with LogTime() as timer:
    for lcl_id in tqdm(lcl_ids):
        for model_config in models_to_run:
            model_config = model_config.clone()
            X_train, y_train, _ = feat_config.get_X_y(
                train_df.loc[train_df.LCLid == lcl_id, :],
                categorical=True,
                exogenous=False,
                test=False
            )
            X_test, y_test, _ = feat_config.get_X_y(
                test_df.loc[test_df.LCLid == lcl_id, :], categorical=True, exogenous=False, test=True
            )
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                y_pred, metrics, feat_df = evaluate_model(
                    model_config,
                    feat_config,
                    missing_value_config,
                    X_train,
                    y_train,
                    X_test,
                    y_test,
                )
            y_pred.name = "predictions"
            y_pred = y_pred.to_frame()
            y_pred["LCLid"] = lcl_id
            y_pred["Algorithm"] = model_config.name
            metrics["LCLid"] = lcl_id
            metrics["Algorithm"] = model_config.name
            y_pred["energy_consumption"] = y_test.values
            all_preds.append(y_pred)
            all_metrics.append(metrics)

  1%|          | 1/185 [00:03<10:23,  3.39s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002151 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5465
[LightGBM] [Info] Number of data points in the train set: 36576, number of used features: 582
[LightGBM] [Info] Start training from score 0.123976


  1%|          | 2/185 [00:06<09:35,  3.14s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5465
[LightGBM] [Info] Number of data points in the train set: 36576, number of used features: 582
[LightGBM] [Info] Start training from score 0.241508


  2%|▏         | 3/185 [00:09<09:36,  3.17s/it]

...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002490 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5459
[LightGBM] [Info] Number of data points in the train set: 31392, number of used features: 579
[LightGBM] [Info] Start training from score 0.165373


 99%|█████████▉| 184/185 [07:28<00:02,  2.48s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002374 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5459
[LightGBM] [Info] Number of data points in the train set: 31392, number of used features: 579
[LightGBM] [Info] Start training from score 0.174888


100%|██████████| 185/185 [07:31<00:00,  2.44s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002303 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5459
[LightGBM] [Info] Number of data points in the train set: 31392, number of used features: 579
[LightGBM] [Info] Start training from score 0.163139
Time Elapsed: 7 minutes and 31 seconds


## Results

In [15]:
pred_df = pd.concat(all_preds)
pred_df.head()

,predictions,LCLid,Algorithm,energy_consumption
timestamp,,,,
2014-02-01 00:00:00,0.074981,MAC000061,Lasso Regression,0.066
2014-02-01 00:30:00,0.076384,MAC000061,Lasso Regression,0.063
2014-02-01 01:00:00,0.070385,MAC000061,Lasso Regression,0.040
2014-02-01 01:30:00,0.055555,MAC000061,Lasso Regression,0.020
2014-02-01 02:00:00,0.039923,MAC000061,Lasso Regression,0.018


In [16]:
metrics_df = pd.DataFrame(all_metrics)
metrics_df.head()

,Algorithm,MAE,MSE,RMSE,Forecast Bias,LCLid
0,Lasso Regression,0.037804,0.003162,0.056235,-1.858942,MAC000061
1,XGB Random Forest,0.035485,0.003077,0.055474,-1.611098,MAC000061
2,LightGBM,0.034493,0.002963,0.054432,0.506766,MAC000061
3,Lasso Regression,0.073742,0.025421,0.159440,-1.785068,MAC000062
4,XGB Random Forest,0.067799,0.025043,0.158251,3.581974,MAC000062


In [17]:
feat_df

,feature,importance
74,energy_consumption_lag_1,419
23,energy_consumption_lag_2,252
75,energy_consumption_lag_334,169
12,day_hourly_profile,148
3,energy_consumption_lag_337,140
...,...,...
209,timestamp_Dayofyear_71,0
206,timestamp_Dayofyear_240,0
204,timestamp_Dayofyear_117,0
203,timestamp_Dayofyear_131,0


In [ ]:
fig = px.bar(feat_df.head(15), x="feature", y="importance")
format_plot(fig, xlabel="Features", ylabel="Importance", font_size=12)
fig.update_layout(title= "LightGBM - Feature Importance")
fig.write_image(dst_img/"lightgbm_reg_fimp.png")
fig.show()

### Lasso

In [19]:
lasso_test_metric_df = pred_df[pred_df.Algorithm =='Lasso Regression']
overall_metrics_test_lasso = {
    "Algorithm": "Lasso Regression",
    "MSE": mse(lasso_test_metric_df.energy_consumption.values, lasso_test_metric_df.predictions.values),
    "RMSE": rmse(lasso_test_metric_df.energy_consumption.values, lasso_test_metric_df.predictions.values),
    "MAE": mae(lasso_test_metric_df.energy_consumption.values, lasso_test_metric_df.predictions.values),
    "Forecast Bias": forecast_bias(lasso_test_metric_df.energy_consumption.values, lasso_test_metric_df.predictions.values)
}
overall_metrics_test_lasso

{'Algorithm': 'Lasso Regression',
 'MSE': 0.02582521488483684,
 'RMSE': 0.16070225538192312,
 'MAE': 0.07751618648925726,
 'Forecast Bias': -1.3277794994791883}

### XGB Random Forest

In [20]:
xgb_test_metric_df = pred_df[pred_df.Algorithm =='XGB Random Forest']
overall_metrics_test_xgb = {
    "Algorithm": "XGB Random Forest",
    "MSE": mse(xgb_test_metric_df.energy_consumption.values, xgb_test_metric_df.predictions.values),
    "RMSE": rmse(xgb_test_metric_df.energy_consumption.values, xgb_test_metric_df.predictions.values),
    "MAE": mae(xgb_test_metric_df.energy_consumption.values, xgb_test_metric_df.predictions.values),
    "Forecast Bias": forecast_bias(xgb_test_metric_df.energy_consumption.values, xgb_test_metric_df.predictions.values)
}
overall_metrics_test_xgb

{'Algorithm': 'XGB Random Forest',
 'MSE': 0.02922888373748025,
 'RMSE': 0.1709645686611125,
 'MAE': 0.07685168621481299,
 'Forecast Bias': 0.42328869670574504}

### LightGBM

In [21]:
gbm_test_metric_df = pred_df[pred_df.Algorithm =='LightGBM']
overall_metrics_test_gbm = {
    "Algorithm": "LightGBM",
    "MSE": mse(gbm_test_metric_df.energy_consumption.values, gbm_test_metric_df.predictions.values),
    "RMSE": rmse(gbm_test_metric_df.energy_consumption.values, gbm_test_metric_df.predictions.values),
    "MAE": mae(gbm_test_metric_df.energy_consumption.values, gbm_test_metric_df.predictions.values),
    "Forecast Bias": forecast_bias(gbm_test_metric_df.energy_consumption.values, gbm_test_metric_df.predictions.values)
}
overall_metrics_test_gbm

{'Algorithm': 'LightGBM',
 'MSE': 0.026510407439444827,
 'RMSE': 0.1628201690192122,
 'MAE': 0.07344330876212404,
 'Forecast Bias': -1.753861233021375}

In [22]:
ml_test_metrics_df = pd.DataFrame([overall_metrics_test_lasso, overall_metrics_test_xgb, overall_metrics_test_gbm])

In [23]:
summary_metrics_df = pd.concat([baseline_metrics_df, ml_test_metrics_df])
summary_metrics_df

,Algorithm,MSE,RMSE,MAE,Forecast Bias
0,AutoETS,0.058671,0.242221,0.119127,-10.707988
1,ARIMA,0.069428,0.263492,0.132783,-13.314743
0,Lasso Regression,0.025825,0.160702,0.077516,-1.327779
1,XGB Random Forest,0.029229,0.170965,0.076852,0.423289
2,LightGBM,0.026510,0.162820,0.073443,-1.753861


## Visualization

In [ ]:
fig = px.histogram(metrics_df,
                   x="MAE",
                   color=metrics_df["Algorithm"],
                   pattern_shape=metrics_df["Algorithm"],
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mae", ylabel="Probability Density")
fig.update_layout(title="Distribution of MAE in the dataset", xaxis_range=[0,1])
fig.write_image(dst_img/"ml_mae_dist.png")
fig.show()

In [ ]:
fig = px.histogram(metrics_df,
                   x="MSE",
                   color=metrics_df["Algorithm"],
                   pattern_shape=metrics_df["Algorithm"],
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mse", ylabel="Probability Density")
fig.update_layout(title="Distribution of MSE in the dataset", xaxis_range=[0,1])
fig.write_image(dst_img/"ml_mse_dist.png")
fig.show()

In [ ]:
fig = px.histogram(metrics_df,
                   x="RMSE",
                   color=metrics_df["Algorithm"],
                   pattern_shape=metrics_df["Algorithm"],
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="rmse", ylabel="Probability Density")
fig.update_layout(title="Distribution of RMSE in the dataset", xaxis_range=[0,1])
fig.write_image(dst_img/"ml_rmse_dist.png")
fig.show()

In [ ]:
fig = px.histogram(metrics_df,
                   x="Forecast Bias",
                   color=metrics_df["Algorithm"],
                   pattern_shape=metrics_df["Algorithm"],
                   marginal="box",
                   nbins=500,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="forecast bias", ylabel="Probability Density")
fig.update_layout(title="Distribution of Forecast Bias in the dataset", xaxis_range=[-15,25])
fig.write_image(dst_img/"ml_bias_dist.png")
fig.show()

In [28]:
display_lclid = "MAC003069"
ts_baseline_test_pred_df = pred_df.loc[pred_df.LCLid==display_lclid]
ts_baseline_test_pred_df = ts_baseline_test_pred_df.reset_index()
ts_aggtest_metrics = metrics_df.loc[metrics_df.LCLid==display_lclid]

In [39]:
lasso_data = ts_baseline_test_pred_df[ts_baseline_test_pred_df['Algorithm'] == 'Lasso Regression'].copy()
xgb_data = ts_baseline_test_pred_df[ts_baseline_test_pred_df['Algorithm'] == 'XGB Random Forest'].copy()

In [ ]:
## Lasso
model_name = ['Lasso Regression']
model_display_name = ['Lasso Regression']

fig = plot_forecast(lasso_data,
                    forecast_columns=['predictions'],
                    forecast_display_names=model_display_name, timestamp_col ='timestamp',
                    target_col_name="energy_consumption")
fig = format_plot(fig)
fig.update_layout(
    title=f"{model_name[0]}: "\
          f"MSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Algorithm==model_name[0]][['MSE']].iloc[0].item():.4f} | "\
          f"RMSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Algorithm==model_name[0]][['RMSE']].iloc[0].item():.4f} | "\
          f"MAE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Algorithm==model_name[0]][['MAE']].iloc[0].item():.4f} | "\
          f"BIAS: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Algorithm==model_name[0]][['Forecast Bias']].iloc[0].item():.4f}"
)
fig.update_xaxes(type="date", range=["2014-02-01", "2014-02-08"])
fig.write_image(dst_img/"lasso.png")
fig.show()

In [ ]:
## XGB Random Forest
model_name = ['XGB Random Forest']
model_display_name = ['XGB Random Forest']

fig = plot_forecast(xgb_data,
                    forecast_columns=['predictions'],
                    forecast_display_names=model_display_name, timestamp_col ='timestamp',
                    target_col_name="energy_consumption")
fig = format_plot(fig)
fig.update_layout(
    title=f"{model_name[0]}: "\
          f"MSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Algorithm==model_name[0]][['MSE']].iloc[0].item():.4f} | "\
          f"RMSE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Algorithm==model_name[0]][['RMSE']].iloc[0].item():.4f} | "\
          f"MAE: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Algorithm==model_name[0]][['MAE']].iloc[0].item():.4f} | "\
          f"BIAS: {ts_aggtest_metrics.loc[ts_aggtest_metrics.Algorithm==model_name[0]][['Forecast Bias']].iloc[0].item():.4f}"
)
fig.update_xaxes(type="date", range=["2014-02-01", "2014-02-08"])
fig.write_image(dst_img/"xgb.png")
fig.show()

In [ ]:
pred_df.to_pickle(dst_result/"ml_test_prediction_df.pkl")
summary_metrics_df.to_pickle(dst_result/"ml_test_metrics_df.pkl")
metrics_df.to_pickle(dst_result/"ml_test_aggregate_metrics.pkl")